In [1]:
import requests
import json
import pprint
import pandas as pd
import pycurl
from itertools import chain
import numpy as np

pp = pprint.PrettyPrinter()

In [2]:
# This is an example of one pull from the balldontlie.io API for a player searched by name
# You can see that lebron james was added as a search term at the end, and we can replicate this with other players
# The goal being to isolate the id's to use later for performance statistics
# Next we will be creating a for loop to get all of the players from our salaries list

In [3]:
# As you can see in this example, any capitalization does not matter for the player search
james = requests.get("https://www.balldontlie.io/api/v1/players/?search=LEbron jaMes")
james.json()

{'data': [{'id': 237,
   'first_name': 'LeBron',
   'height_feet': 6,
   'height_inches': 8,
   'last_name': 'James',
   'position': 'F',
   'team': {'id': 14,
    'abbreviation': 'LAL',
    'city': 'Los Angeles',
    'conference': 'West',
    'division': 'Pacific',
    'full_name': 'Los Angeles Lakers',
    'name': 'Lakers'},
   'weight_pounds': 250}],
 'meta': {'total_pages': 1,
  'current_page': 1,
  'next_page': None,
  'per_page': 25,
  'total_count': 1}}

In [4]:
# We take the file from BDLio and convert it to a dictionary so we car parse the ID
james_dict = json.loads(james.text)
print(james_dict)
type(james_dict)

{'data': [{'id': 237, 'first_name': 'LeBron', 'height_feet': 6, 'height_inches': 8, 'last_name': 'James', 'position': 'F', 'team': {'id': 14, 'abbreviation': 'LAL', 'city': 'Los Angeles', 'conference': 'West', 'division': 'Pacific', 'full_name': 'Los Angeles Lakers', 'name': 'Lakers'}, 'weight_pounds': 250}], 'meta': {'total_pages': 1, 'current_page': 1, 'next_page': None, 'per_page': 25, 'total_count': 1}}


dict

In [5]:
james_id = james_dict['data'][0]["id"]

In [6]:
%store -r player_names_list
player_names_list

['Stephen Curry',
 'John Wall',
 'Russell Westbrook',
 'James Harden',
 'Damian Lillard',
 'LeBron James',
 'Kevin Durant',
 'Giannis Antetokounmpo',
 'Paul George',
 'Kawhi Leonard',
 'Klay Thompson',
 'Jimmy Butler',
 'Tobias Harris',
 'Khris Middleton',
 'Anthony Davis',
 'Rudy Gobert',
 'Kyrie Irving',
 'Bradley Beal',
 'Kristaps Porzingis',
 'Devin Booker']

In [7]:
url = "https://www.balldontlie.io/api/v1/players/?search="
player_id_list = []

for name in player_names_list:
    player_url = url + f"{name}"
    player_get = requests.get(player_url)
    player_get.json()
    player_dict = json.loads(player_get.text)
    player_id_list.append(player_dict['data'][0]["id"])
print(player_id_list)
%store player_id_list

[115, 1014, 472, 192, 278, 237, 140, 15, 172, 274, 443, 79, 200, 315, 117, 176, 228, 37, 378, 57]
Stored 'player_id_list' (list)


In [8]:
averages_url = "https://www.balldontlie.io/api/v1/season_averages?player_ids[]="
averages_list = []
for ids in player_id_list:
    averages_new = averages_url + f"{str(ids)}"
    averages_get = requests.get(averages_new)
    averages_get.json()
    averages_dict = json.loads(averages_get.text)
    averages_list.append(averages_dict['data'])
    if ids == 57:
        break
averages_list
  

[[{'games_played': 15,
   'player_id': 115,
   'season': 2021,
   'min': '32:35',
   'fgm': 8.87,
   'fga': 19.53,
   'fg3m': 5.27,
   'fg3a': 12.87,
   'ftm': 4.67,
   'fta': 4.87,
   'oreb': 0.67,
   'dreb': 5.53,
   'reb': 6.2,
   'ast': 6.47,
   'stl': 1.73,
   'blk': 0.6,
   'turnover': 3.27,
   'pf': 1.6,
   'pts': 27.67,
   'fg_pct': 0.454,
   'fg3_pct': 0.409,
   'ft_pct': 0.959}],
 [],
 [{'games_played': 16,
   'player_id': 472,
   'season': 2021,
   'min': '34:13',
   'fgm': 7.25,
   'fga': 16.75,
   'fg3m': 1.31,
   'fg3a': 4.19,
   'ftm': 3.25,
   'fta': 4.69,
   'oreb': 1.38,
   'dreb': 7.0,
   'reb': 8.38,
   'ast': 8.19,
   'stl': 1.38,
   'blk': 0.19,
   'turnover': 5.25,
   'pf': 3.06,
   'pts': 19.06,
   'fg_pct': 0.433,
   'fg3_pct': 0.313,
   'ft_pct': 0.693}],
 [{'games_played': 16,
   'player_id': 192,
   'season': 2021,
   'min': '33:11',
   'fgm': 5.75,
   'fga': 13.44,
   'fg3m': 2.88,
   'fg3a': 7.25,
   'ftm': 5.19,
   'fta': 5.94,
   'oreb': 0.94,
   'dreb':

In [9]:
averages_df = pd.DataFrame(list(chain.from_iterable(averages_list)))
averages_df

,games_played,player_id,season,min,fgm,fga,fg3m,fg3a,ftm,fta,...,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct
0,15,115,2021,32:35,8.87,19.53,5.27,12.87,4.67,4.87,...,6.20,6.47,1.73,0.60,3.27,1.60,27.67,0.454,0.409,0.959
1,16,472,2021,34:13,7.25,16.75,1.31,4.19,3.25,4.69,...,8.38,8.19,1.38,0.19,5.25,3.06,19.06,0.433,0.313,0.693
2,16,192,2021,33:11,5.75,13.44,2.88,7.25,5.19,5.94,...,7.25,8.63,1.13,0.69,4.63,2.19,19.56,0.428,0.397,0.874
3,15,278,2021,34:18,7.40,18.60,2.73,9.07,2.67,3.13,...,4.27,7.67,0.67,0.20,2.47,1.47,20.20,0.398,0.301,0.851
4,6,237,2021,37:03,9.50,20.33,2.83,8.17,3.00,3.83,...,5.50,7.00,2.33,0.83,4.17,2.33,24.83,0.467,0.347,0.783
5,16,140,2021,33:27,10.44,18.25,2.00,4.63,5.44,6.44,...,7.69,4.69,0.50,0.50,2.94,1.13,28.31,0.572,0.432,0.845
6,14,15,2021,31:45,9.64,18.86,1.36,4.50,6.00,8.86,...,10.86,5.71,1.07,1.64,2.93,2.93,26.64,0.511,0.302,0.677
7,14,172,2021,35:02,9.79,21.93,3.00,8.64,4.43,4.93,...,8.29,5.07,2.21,0.43,4.43,2.57,27.00,0.446,0.347,0.899
8,12,79,2021,31:51,8.00,14.92,0.50,1.50,6.92,8.25,...,5.42,4.92,1.92,0.42,1.83,1.83,23.42,0.536,0.333,0.838
9,9,200,2021,33:01,8.00,15.89,1.44,4.22,2.78,3.00,...,8.44,4.44,0.11,0.56,1.89,1.56,20.22,0.503,0.342,0.926


In [10]:
%store -r salary_df


In [11]:
player_id_df = pd.DataFrame(player_id_list)
salary_id_join = player_id_df.join(salary_df)

In [12]:
salary_player_id_df = salary_id_join.rename(columns={0: 'player_id'})
salary_player_id_df

,player_id,Rk,Player,Tm,2021-22
0,115,1,Stephen Curry,GSW,$45780966.00
1,1014,2,John Wall,HOU,$44310840.00
2,472,3,Russell Westbrook,LAL,$44211146.00
3,192,4,James Harden,BRK,$43848000.00
4,278,5,Damian Lillard,POR,$43750000.00
5,237,6,LeBron James,LAL,$41180544.00
6,140,7,Kevin Durant,BRK,$40918900.00
7,15,8,Giannis Antetokounmpo,MIL,$39344970.00
8,172,9,Paul George,LAC,$39344970.00
9,274,10,Kawhi Leonard,LAC,$39344900.00


In [13]:
player_mapper = dict(zip(salary_player_id_df['player_id'].values, salary_player_id_df["Player"].values))
player_mapper
%store player_mapper

Stored 'player_mapper' (dict)


In [14]:
all_data_df = salary_player_id_df.merge(averages_df, on='player_id', how = 'outer')
all_data_df.set_index('Rk').fillna(0)

,player_id,Player,Tm,2021-22,games_played,season,min,fgm,fga,fg3m,...,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct
Rk,,,,,,,,,,,,,,,,,,,,,
1,115,Stephen Curry,GSW,$45780966.00,15.0,2021.0,32:35,8.87,19.53,5.27,...,6.20,6.47,1.73,0.60,3.27,1.60,27.67,0.454,0.409,0.959
2,1014,John Wall,HOU,$44310840.00,0.0,0.0,0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000
3,472,Russell Westbrook,LAL,$44211146.00,16.0,2021.0,34:13,7.25,16.75,1.31,...,8.38,8.19,1.38,0.19,5.25,3.06,19.06,0.433,0.313,0.693
4,192,James Harden,BRK,$43848000.00,16.0,2021.0,33:11,5.75,13.44,2.88,...,7.25,8.63,1.13,0.69,4.63,2.19,19.56,0.428,0.397,0.874
5,278,Damian Lillard,POR,$43750000.00,15.0,2021.0,34:18,7.40,18.60,2.73,...,4.27,7.67,0.67,0.20,2.47,1.47,20.20,0.398,0.301,0.851
6,237,LeBron James,LAL,$41180544.00,6.0,2021.0,37:03,9.50,20.33,2.83,...,5.50,7.00,2.33,0.83,4.17,2.33,24.83,0.467,0.347,0.783
7,140,Kevin Durant,BRK,$40918900.00,16.0,2021.0,33:27,10.44,18.25,2.00,...,7.69,4.69,0.50,0.50,2.94,1.13,28.31,0.572,0.432,0.845
8,15,Giannis Antetokounmpo,MIL,$39344970.00,14.0,2021.0,31:45,9.64,18.86,1.36,...,10.86,5.71,1.07,1.64,2.93,2.93,26.64,0.511,0.302,0.677
9,172,Paul George,LAC,$39344970.00,14.0,2021.0,35:02,9.79,21.93,3.00,...,8.29,5.07,2.21,0.43,4.43,2.57,27.00,0.446,0.347,0.899


In [15]:
# Game Score = Points Scored + (0.4 x Field Goals) – (0.7 x Field Goal Attempts) – (0.4 x (Free Throw Attempts – Free Throws)) + 
# (0.7 x Offensive Rebounds) + (0.3 x Defensive Rebounds) + Steals + (0.7 x Assists) + (0.7 x Blocks) – (0.4 x Personal Fouls) – Turnovers

In [16]:
game_scores_list = []
for player in range (len(all_data_df)):
    a = all_data_df.loc[player, 'pts']
    b = all_data_df.loc[player, 'fgm'] * 0.4
    c = all_data_df.loc[player,'fga'] * 0.7
    d = ((all_data_df.loc[player,'fta']) - (all_data_df.loc[player, 'ftm']) * 0.4)
    e = all_data_df.loc[player,'oreb'] * 0.7
    f = all_data_df.loc[player,'dreb'] * 0.3
    g = all_data_df.loc[player,'stl']
    h = all_data_df.loc[player,'ast'] * 0.7
    i = all_data_df.loc[player,'blk'] * 0.7
    j = all_data_df.loc[player,'pf'] * 0.4
    k = all_data_df.loc[player,'turnover']
    game_score_player = a + b - c - d + e + f + g + h + i - j - k
    game_scores_list.append([game_score_player])
game_scores_list

[[19.442000000000007],
 [nan],
 [10.683],
 [13.286999999999999],
 [12.64],
 [16.328000000000003],
 [18.671000000000003],
 [16.886000000000003],
 [15.721],
 [nan],
 [nan],
 [16.045],
 [14.524999999999999],
 [9.027],
 [16.117],
 [11.575000000000003],
 [nan],
 [11.548000000000004],
 [12.018],
 [12.176]]

In [17]:
game_score_df = pd.DataFrame(game_scores_list)
game_score_df.fillna(0)


,0
0,19.442
1,0.000
2,10.683
3,13.287
4,12.640
5,16.328
6,18.671
7,16.886
8,15.721
9,0.000


In [18]:
salaries_df = all_data_df.iloc[: , 4].str.replace('$', '')
salaries_df.astype(float)

/Users/tgmyers/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


0     45780966.0
1     44310840.0
2     44211146.0
3     43848000.0
4     43750000.0
5     41180544.0
6     40918900.0
7     39344970.0
8     39344970.0
9     39344900.0
10    37980720.0
11    36016200.0
12    36000000.0
13    35500000.0
14    35361360.0
15    35344828.0
16    34916200.0
17    34502130.0
18    31650600.0
19    31610000.0
Name: 2021-22, dtype: float64

In [19]:
salaries_gamescore_nan = pd.merge(salaries_df, game_score_df, right_index=True, left_index=True)
salaries_gamescore_df = salaries_gamescore_nan.fillna(0)
salaries_gamescore_df['2021-22'] = salaries_gamescore_df['2021-22'].astype(float)
salaries_gamescore_df.dtypes

2021-22    float64
0          float64
dtype: object

In [20]:
salaries_gamescore_df = salaries_gamescore_df.rename(columns = {0 : 'game_score'})
salaries_gamescore_df

,2021-22,game_score
0,45780966.0,19.442
1,44310840.0,0.000
2,44211146.0,10.683
3,43848000.0,13.287
4,43750000.0,12.640
5,41180544.0,16.328
6,40918900.0,18.671
7,39344970.0,16.886
8,39344970.0,15.721
9,39344900.0,0.000


In [38]:
# Create the GameScore ratio function and add it to a new dataframe
pd.options.display.float_format = '{:,.2f}'.format
salaries_gamescore_df['GameScore_Salary_Ratio'] = (salaries_gamescore_df['game_score'] * 10000000) / salaries_gamescore_df["2021-22"]
salaries_gamescore_df['Player Name'] = player_names_list
salaries_gamescore_df

,2021-22,game_score,GameScore_Salary_Ratio,Player Name
0,"45,780,966.00",19.44,4.25,Stephen Curry
1,"44,310,840.00",0.00,0.00,John Wall
2,"44,211,146.00",10.68,2.42,Russell Westbrook
3,"43,848,000.00",13.29,3.03,James Harden
4,"43,750,000.00",12.64,2.89,Damian Lillard
5,"41,180,544.00",16.33,3.96,LeBron James
6,"40,918,900.00",18.67,4.56,Kevin Durant
7,"39,344,970.00",16.89,4.29,Giannis Antetokounmpo
8,"39,344,970.00",15.72,4.00,Paul George
9,"39,344,900.00",0.00,0.00,Kawhi Leonard
